In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [11]:
x_train = train.drop(['id', 'digit', 'letter'], axis=1).values
# x_train, x_train.shape

# x_train을 keras input에 넣기 위해 4차원으로 변경. 
# 28은 784개의 1차원벡터로 된 특성을 2차원으로 바꿔주기 위함. 784=28*28
x_train = x_train.reshape(-1, 28, 28, 1)
# type(x_train), np.max(x_train)
x_train = x_train/255

y = train['digit']
y_train = np.zeros((len(y), len(y.unique())))

# 원 핫 벡터로 
for i, digit in enumerate(y):
    y_train[i, digit] = 1

In [12]:
def create_cnn_model(x_train):
    inputs = tf.keras.layers.Input(x_train.shape[1:])

    bn = tf.keras.layers.BatchNormalization()(inputs)
    conv = tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

    bn = tf.keras.layers.BatchNormalization()(pool)
    conv = tf.keras.layers.Conv2D(256, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(256, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

    flatten = tf.keras.layers.Flatten()(pool)

    bn = tf.keras.layers.BatchNormalization()(flatten)
    dense = tf.keras.layers.Dense(1000, activation='relu')(bn)

    bn = tf.keras.layers.BatchNormalization()(dense)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(bn)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return model

In [13]:
model = create_cnn_model(x_train)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
64/64 [==============================] - 16s 251ms/step - loss: 2.6003 - accuracy: 0.4609
Epoch 2/20
64/64 [==============================] - 16s 255ms/step - loss: 0.8147 - accuracy: 0.7222
Epoch 3/20
64/64 [==============================] - 17s 265ms/step - loss: 0.4393 - accuracy: 0.8472
Epoch 4/20
64/64 [==============================] - 16s 252ms/step - loss: 0.2881 - accuracy: 0.9043
Epoch 5/20
64/64 [==============================] - 16s 250ms/step - loss: 0.2182 - accuracy: 0.9287
Epoch 6/20
64/64 [==============================] - 16s 254ms/step - loss: 0.1353 - accuracy: 0.9502
Epoch 7/20
64/64 [==============================] - 16s 248ms/step - loss: 0.1097 - accuracy: 0.9604
Epoch 8/20
64/64 [==============================] - 16s 250ms/step - loss: 0.1139 - accuracy: 0.9639
Epoch 9/20
64/64 [==============================] - 16s 246ms/step - loss: 0.0790 - accuracy: 0.9761
Epoch 10/20
64/64 [==============================] - 16s 251ms/step - loss: 0.0787 - accura

In [31]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
y=model.predict(x_test)

In [32]:
# y.shape

In [33]:
digit=np.argmax(y,axis=1)
# number

In [41]:
pd_submisson=pd.DataFrame({"digit":digit},index=test["id"])
pd_submisson.index

Int64Index([ 2049,  2050,  2051,  2052,  2053,  2054,  2055,  2056,  2057,
             2058,
            ...
            22519, 22520, 22521, 22522, 22523, 22524, 22525, 22526, 22527,
            22528],
           dtype='int64', name='id', length=20480)

In [42]:
pd_submisson

,digit
id,
2049,6
2050,9
2051,6
2052,0
2053,3
...,...
22524,4
22525,1
22526,6


In [46]:
pd_submisson.to_csv("baseline.csv", encoding="utf-8")